In [3]:
#from bs4 import BeautifulSoup
import re
import xml.etree.ElementTree as ET
from attribute import Attribute as att
from copy import deepcopy

In [4]:
my_parser = ET.XMLParser(encoding='utf-8')
tree = ET.parse('dicom-2020a.dicomextract.txt', parser=my_parser)
root = tree.getroot()

In [5]:
c=[]
c = root.findall(".//ItemEntry[@Type='1C']") + root.findall(".//ItemEntry[@Type='2C']")

In [6]:
#Dictionary with conditional attributes:
attribute_dict = []
for a in c:
    description = a.attrib['Description']
    tag = a.attrib['Tag']
    type = a.attrib['Type']
    attribute_dicom = att(description,tag, type)
    attribute_dict.append(attribute_dicom)
attribute_original = deepcopy(attribute_dict)

In [7]:
for a in attribute_dict:
    a.set_only_rule_def()
#print(attribute_original[2].description)
for i in attribute_dict:
    print(i.description, '\n')

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required only if (0004,1511) is absent (see Note). 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if the IOD of the Presentation SOP Instance referenced by this directory record includes the Presentation State Relationship Module (PS3.3:sect_C.11.11). 

Required if the SOP Instance referenced 

In [8]:
# Remove Notes, See Section, May be present
#print(attribute_original[60].description)
for a in attribute_dict:
    a.clean_note()
    a.clean_section()
    a.clean_may_be_present()
    a.clean_num_items_present()
#print(attribute_dict[60].description)
for i in attribute_dict:
    print(i.description, '\n')


Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required only if (0004,1511) is absent (see  

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if the IOD of the Presentation SOP Instance referenced by this directory record includes the Presentation State Relationship Module (PS3.3:sect_C.11.11). 

Required if the SOP Instance referenced by thi

In [9]:
# for a in attribute_original:
#     if a.tag=='(0054,0414)':
#         print(a.description, '\n')
#         description = a.description
# for a in attribute_dict:
#     if a.tag=='(0054,0414)':
#         print(a.description)
#300A,0140
description = 'Direction of Table Top Eccentric Rotation when viewing table from above, for segment following Control Point. Required for first Item of Control Point Sequence, or if Table Top Eccentric Rotation Direction changes during Beam. See Section C.8.8.14.8 (PS3.3:sect_C.8.8.14.8). Enumerated Values: CW clockwise CC counter-clockwise NONE no rotation'
if_req = re.search('(if required).*(shall)[^.]*[.]', description, re.IGNORECASE)
section = re.search('((Required)((.*(Enumerated values).*$)|[^.]*[.]\\s|[^.]*$|.*[.]$))',
                            description, re.IGNORECASE)
s = re.search('(See Section)(([^\\)]*[)][^.]*[.])|([^\\)]*[)][^\\)]*[\\)][.])|'
                            '([^\\)]*[))])|([^\\)]*[)][^.]*$))', section.group(), re.IGNORECASE)
print(s.group())

See Section C.8.8.14.8 (PS3.3:sect_C.8.8.14.8).


In [12]:
for a in attribute_dict:
    if 'Enumerated Values' in a.description:
        print(a.description, '\n')

Required if Universal Entity ID (0040,0032) is present. Enumerated Values: DNS An Internet dotted name. Either in ASCII or as integers EUI64 An IEEE Extended Unique Identifier ISO An International Standards Organization Object Identifier URI Uniform Resource Identifier UUID The DCE Universal Unique Identifier X400 An X.400 MHS identifier X500 An X.500 directory name 

Required if Frame Type (0008,9007) Value 1 of this frame is ORIGINAL.  Enumerated Values: YES quadrature or circularly polarized NO linear 

Required if Multi-frame Image. Contains the Data Element Tag of the Attribute that is used as the Frame increment in Multi-frame image pixel data ( Enumerated Values: 00181063H Frame Time (0018,1063) 00181065H Frame Time Vector (0018,1065) 

Required if Pixel Intensity Relationship (0028,1040) is present. Enumerated Values: +1 Lower pixel values correspond to less X-Ray beam intensity -1 Higher pixel values correspond to less X-Ray beam intensity  

Required for first Item of Control

In [11]:
d = 'The IOL power that would be required to achieve exact emmetropia, or no need for glasses at distance after surgery, in diopters. If Type of Optical Correction (0022,1046) is TORIC, this value represents the spherical equivalent of the toric intraocular lens power for exact emmetropia.'
req = re.search('((Required)((.*(Enumerated values).*$)|.*(Section).*$|[^.*]*[.]))', d, re.IGNORECASE)
print(req.groups())

('required to achieve exact emmetropia, or no need for glasses at distance after surgery, in diopters.', 'required', ' to achieve exact emmetropia, or no need for glasses at distance after surgery, in diopters.', None, None, None)
